## <font color=blue>Check Individual Module *COUPLING* <font color=green> **local** </font>  Results</font>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import netCDF4 as nc
from pandas.plotting import table 
import xarray as xr

#### <font color=blue>List of *Modules* and <font color=green> **Datasets** </font> to be used </font>

In [3]:
# Load FACTS workflow data as a dataframe.
df = pd.read_fwf('../001_GMD_IP_Modules_Data/ModuleTest_DataFile_List_coupling.local.txt',comment = '#')
df

,Component,Module,sub_Module,DataFile
0,TE,tlm,sterodynamics,ocean.tlm.sterodynamics_localsl.nc
1,Glaciers,ipccar5,glaciers,ar5glaciers.ipccar5.glaciers_localsl.nc
2,Glaciers,emulandice,glaciers,emuglaciers.emulandice.glaciers_localsl.nc
3,GrIS,bamber19,icesheets,bamber19.bamber19.icesheets_GIS_localsl.nc
4,GrIS,FittedISMIP,GrIS,GrIS1f.FittedISMIP.GrIS_GIS_localsl.nc
5,GrIS,emulandice,GrIS,emuGrIS.emulandice.GrIS_localsl.nc
6,AIS,bamber19,icesheets,bamber19.bamber19.icesheets_AIS_localsl.nc
7,AIS,deconto21,AIS,deconto21.deconto21.AIS_AIS_localsl.nc
8,AIS,emulandice,AIS,emuAIS.emulandice.AIS_localsl.nc
9,AIS,larmip,AIS,larmip.larmip.AIS_localsl.nc


#### <font color=blue> **OPTIONAL USE**: <font color=green> *Write nc data details to a file* </font>

In [4]:
EXPDATE     = 221217
MODNO       = df.index.values;   # Number of datasets Module outputs
SCENARIO    = ['ssp126', 'ssp245', 'ssp585']; 
#        
file = open("ModuleData_Info_COUPLING.local-{arg1}.txt".format(arg1=EXPDATE), "w")
#
for val in MODNO: 
    for scenario in SCENARIO:
        #
        MODULE = df["Module"][val];  SUB_MODULE = df["sub_Module"][val]; COMP = df["Component"][val];
        # Skip if data is absent.
        if df["DataFile"][val] == 'XXX':
           continue 
        #
        # Pick the data files & Import nc file to dataframe. 
        dataFOLDER  = '/projects/kopp/facts-experiments/{arg2}/coupling.{arg1}/output/'.format(arg1=scenario,arg2=EXPDATE)
        dataFILE    = 'coupling.{arg1}.'.format(arg1=scenario) + df["DataFile"][val]
        d_nc        = xr.open_dataset(dataFOLDER + dataFILE)
        d_nc
    file.write( "DataFILE ==> "+str(df["DataFile"][val])+"\n"+ str(d_nc.dims) + "\n" + str(d_nc.data_vars)+ "\n" + str(d_nc.coords)+"\n"+"-------------"+"\n"+"\n" )
        #
file.close()


#### $\color{Red}{\text{Load Module data from ==> facts-experiments}}$

In [5]:
EXPDATE     = 221217
MODNO       = df.index.values;   # Number of datasets Module outputs
SCENARIO    = ['ssp126', 'ssp245', 'ssp585'];
#SCENARIO    = ['ssp119','ssp126', 'ssp245', 'ssp370', 'ssp585'];
YEAR        = 2100; # Percentile Year for the table.    
#
# Temp DF to store all .nc O/P.      
df_TEMP      = []
#        
for scenario in SCENARIO:
    # Define pd DF columns to append.
    PERCENTILE  = '%ile({arg2})'.format(arg2=scenario)
    percentile = np.nan
    df[PERCENTILE] = percentile
    df[PERCENTILE] = df[PERCENTILE].astype('object') 
    #
    for val in MODNO: 
        #
        MODULE = df["Module"][val];  SUB_MODULE = df["sub_Module"][val]; COMP = df["Component"][val]; DATAFILE = df["DataFile"][val];
        # Skip if data is absent.
        if df["DataFile"][val] == 'XXX':
           continue 
        #
        # Pick the data files & Import nc file to dataframe. 
        dataFOLDER  = '/projects/kopp/facts-experiments/{arg2}/coupling.{arg1}/output/'.format(arg1=scenario,arg2=EXPDATE)
        dataFILE    = 'coupling.{arg1}.'.format(arg1=scenario) + df["DataFile"][val]
        d_nc        = xr.open_dataset(dataFOLDER + dataFILE)
        d_nc
        #
        # Percentile calculation.
        percentList = [50, 17, 83]          #[50, 5, 17, 83, 95]
        # Find year index to pick SLC value.
        Yind    = np.where(d_nc["years"].values == YEAR)[0][0] 
        GMSL    = (d_nc["sea_level_change"][:,Yind,0].values)/1       
        # Find Percentiles.
        pcntle = np.percentile(GMSL[:], percentList );    pcntle = np.around(pcntle,2)
        #
        # df.at[val,PERCENTILE] = [pcntle[0], pcntle[1],pcntle[2]]
        df.at[val,PERCENTILE] = f'{pcntle[0]:2.2f} ({pcntle[1]:2.2f} , {pcntle[2]:2.2f})' 
        # Print the OP.
print('GMSL Component Projections for ',d_nc["years"][Yind].values, 'at location ID', d_nc["locations"].values)    
#show(df)
df

GMSL Component Projections for  2100 at location ID [12]


,Component,Module,sub_Module,DataFile,%ile(ssp126),%ile(ssp245),%ile(ssp585)
0,TE,tlm,sterodynamics,ocean.tlm.sterodynamics_localsl.nc,"240.00 (92.00 , 394.00)","343.00 (196.83 , 489.00)","494.00 (338.83 , 653.17)"
1,Glaciers,ipccar5,glaciers,ar5glaciers.ipccar5.glaciers_localsl.nc,"73.00 (53.00 , 99.00)","92.00 (66.83 , 125.00)","128.00 (90.00 , 174.00)"
2,Glaciers,emulandice,glaciers,emuglaciers.emulandice.glaciers_localsl.nc,"75.00 (57.00 , 94.00)","101.00 (81.00 , 120.00)","145.00 (123.00 , 166.00)"
3,GrIS,bamber19,icesheets,bamber19.bamber19.icesheets_GIS_localsl.nc,"56.00 (31.00 , 134.00)","56.00 (31.00 , 134.00)","98.00 (45.00 , 259.17)"
4,GrIS,FittedISMIP,GrIS,GrIS1f.FittedISMIP.GrIS_GIS_localsl.nc,"33.00 (26.00 , 43.00)","44.00 (35.00 , 55.00)","61.00 (49.00 , 77.17)"
5,GrIS,emulandice,GrIS,emuGrIS.emulandice.GrIS_localsl.nc,"24.00 (5.00 , 46.00)","34.00 (15.83 , 56.00)","55.00 (36.00 , 78.00)"
6,AIS,bamber19,icesheets,bamber19.bamber19.icesheets_AIS_localsl.nc,"117.00 (-6.00 , 307.17)","117.00 (-6.00 , 307.17)","241.00 (28.00 , 684.17)"
7,AIS,deconto21,AIS,deconto21.deconto21.AIS_AIS_localsl.nc,"107.00 (83.00 , 137.00)","117.00 (92.00 , 144.00)","385.00 (229.00 , 596.00)"
8,AIS,emulandice,AIS,emuAIS.emulandice.AIS_localsl.nc,"98.00 (35.00 , 166.00)","95.00 (33.00 , 163.00)","99.00 (35.00 , 164.00)"
9,AIS,larmip,AIS,larmip.larmip.AIS_localsl.nc,"150.50 (63.00 , 305.00)","158.00 (65.00 , 333.00)","177.00 (64.00 , 397.00)"


In [ ]:
df.to_excel(r'Table_CheckModule_COUPLING_local_GMSL-Proj_2100-{arg1}.xlsx'.format(arg1=EXPDATE), index=False)


In [17]:
# View DatasetDetais
DUM0    = "ocean.tlm.sterodynamics_localsl.nc"
DUM1    = '/projects/kopp/facts-experiments/{arg2}/coupling.{arg1}/output/'.format(arg1=SCENARIO[0],arg2=EXPDATE)
DUM2    = 'coupling.{arg1}.'.format(arg1=SCENARIO[0]) + DUM0
# DUM_nc  = xr.open_dataset(DUM1 + DUM2)
# DUM_nc